In [3]:
#libraries importing
import pandas as pd

In [62]:
#reading the NYT_API data
NYT_df=pd.read_csv("output/best_book_list_1.csv")
NYT_df.head()

,rank,primary_isbn10,primary_isbn13,publisher,title,author,category,pub_date
0,1,0593356152,9780593356159,Ballantine,THE MAID,Nita Prose,Hardcover Fiction,current
1,2,0525559477,9780525559474,Viking,THE MIDNIGHT LIBRARY,Matt Haig,Hardcover Fiction,current
2,3,0735222355,9780735222359,Viking,THE LINCOLN HIGHWAY,Amor Towles,Hardcover Fiction,current
3,4,0316499773,9780316499774,"Little, Brown",THE HORSEWOMAN,James Patterson and Mike Lupica,Hardcover Fiction,current
4,5,1501171348,9781501171345,Simon & Schuster,THE LAST THING HE TOLD ME,Laura Dave,Hardcover Fiction,current


In [63]:
#checking columns and null values
NYT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3640 entries, 0 to 3639
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rank            3640 non-null   int64 
 1   primary_isbn10  3067 non-null   object
 2   primary_isbn13  3640 non-null   object
 3   publisher       3640 non-null   object
 4   title           3640 non-null   object
 5   author          3640 non-null   object
 6   category        3640 non-null   object
 7   pub_date        3640 non-null   object
dtypes: int64(1), object(7)
memory usage: 227.6+ KB


In [64]:
#dropping column with null values
NYT_df.drop('primary_isbn10',axis=1,inplace=True)

In [65]:
#checking duplicated rows
NYT_df.duplicated().sum()

0

In [66]:
NYT_df["category"].value_counts()

Combined Print Nonfiction    560
Combined Print Fiction       560
Hardcover Nonfiction         420
Audio Nonfiction             420
E-Book Fiction               420
Audio Fiction                420
Hardcover Fiction            420
E-Book Nonfiction            420
Name: category, dtype: int64

In [67]:
# checking the unique number of title column
print(f'The unique number of title column: {NYT_df["title"].nunique()}')
NYT_df["title"].value_counts().to_frame()

The unique number of title column: 1070


,title
THE MIDNIGHT LIBRARY,40
UNTAMED,38
THE LAST THING HE TOLD ME,34
GREENLIGHTS,34
BORN A CRIME,31
...,...
THE LINCOLN CONSPIRACY,1
KILLING CRAZY HORSE,1
THE PIGEON TUNNEL,1
THE PERFECT HORSE,1


In [69]:
# changing the pub_date column to datetime format and sorting dataframe based on the pub_date
NYT_df.loc[NYT_df['pub_date']=='current','pub_date']="2022-02-02"
NYT_df['pub_date'] = pd.to_datetime(NYT_df['pub_date'])
NYT_df=NYT_df.sort_values(by='pub_date',ascending=False)

In [70]:
#drop duplicated rows with the same columns of 'primary_isbn13','publisher','title','author','category':
NYT_df.drop_duplicates(subset=['primary_isbn13','publisher','title','author','category'],inplace=True)

In [71]:
NYT_df.loc[NYT_df['title']=='THE STORYTELLER']

,rank,primary_isbn13,publisher,title,author,category,pub_date
442,8,9780063076099,Dey Street,THE STORYTELLER,Dave Grohl,Hardcover Nonfiction,2022-02-06
2104,5,9780063076129,HarperAudio,THE STORYTELLER,Dave Grohl,Audio Nonfiction,2022-02-02
645,1,9780063076099,‎Dey Street,THE STORYTELLER,Dave Grohl,Hardcover Nonfiction,2021-10-31
2616,17,9781439102763,Atria/Emily Bestler Books,THE STORYTELLER,Jodi Picoult,Combined Print Fiction,2013-04-14


In [72]:
NYT_df['category'].value_counts()

E-Book Fiction               241
Audio Fiction                187
Combined Print Fiction       170
E-Book Nonfiction            148
Audio Nonfiction             147
Combined Print Nonfiction    129
Hardcover Nonfiction         112
Hardcover Fiction            103
Name: category, dtype: int64

In [74]:
NYT_df.to_csv("output/clean_NYT.csv",index=False)

In [30]:
#importing dependencies
import requests
import time
from pprint import pprint
import numpy as np

# Import API key
from config import api_key

In [4]:
#reading in the NYT csv file
nyt_df = pd.read_csv('output/clean_NYT.csv')

nyt_df.head()

,rank,primary_isbn13,publisher,title,author,category,pub_date
0,11,9781982168018,Gallery,TASTE,Stanley Tucci,Hardcover Nonfiction,2022-02-06
1,8,9780062888341,Harper,THE STRANGER IN THE LIFEBOAT,Mitch Albom,Hardcover Fiction,2022-02-06
2,10,9780385547932,Doubleday,TO PARADISE,Hanya Yanagihara,Hardcover Fiction,2022-02-06
3,11,9780593465066,Viking,CALL US WHAT WE CARRY,Amanda Gorman,Hardcover Fiction,2022-02-06
4,12,9781982168438,Scribner,CLOUD CUCKOO LAND,Anthony Doerr,Hardcover Fiction,2022-02-06


In [9]:
#setting up additional columns to hold information
nyt_df['Amazon Product ID'] = ""
nyt_df['Amazon Rating'] = ""
nyt_df['Amazon Rating Total'] = ""
nyt_df['Amazon Price'] = ""

nyt_df.head()

,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price
0,11,9781982168018,Gallery,TASTE,Stanley Tucci,Hardcover Nonfiction,2022-02-06,,,,
1,8,9780062888341,Harper,THE STRANGER IN THE LIFEBOAT,Mitch Albom,Hardcover Fiction,2022-02-06,,,,
2,10,9780385547932,Doubleday,TO PARADISE,Hanya Yanagihara,Hardcover Fiction,2022-02-06,,,,
3,11,9780593465066,Viking,CALL US WHAT WE CARRY,Amanda Gorman,Hardcover Fiction,2022-02-06,,,,
4,12,9781982168438,Scribner,CLOUD CUCKOO LAND,Anthony Doerr,Hardcover Fiction,2022-02-06,,,,


In [10]:
#declaring the parameters
params = {
    'api_key': api_key,
    'type': 'search',
    'amazon_domain': 'amazon.com'
    }

#using iterrows to iterate throguh pandas dataframe
for index, row in nyt_df.iterrows():
    #obtaining the ISBN of the books
    isbn = row["primary_isbn13"]
        
    #adding ISBN search_term to params
    params['search_term'] = isbn    

    #making the url for requests 
    print(f"Retrieving Results for {index} : {isbn}.")
    response = requests.get('https://api.rainforestapi.com/request', params)
    
    #converting response into json
    response=response.json()

    #try to grab the following information, if available
    try:
        nyt_df.loc[index, 'Amazon Product ID'] = response["search_results"][0]["asin"]
        nyt_df.loc[index, 'Amazon Rating'] = response["search_results"][0]["rating"]
        nyt_df.loc[index, 'Amazon Rating Total'] = response["search_results"][0]["ratings_total"]
        nyt_df.loc[index, 'Amazon Price'] = response["search_results"][0]["prices"][0]["value"]
           
    #including a condition for handling books that are not listed in Amazon
    except (KeyError, IndexError):
        print(f"Title not found. Skipping...")

Retrieving Results for 0 : 9781982168018.
Retrieving Results for 1 : 9780062888341.
Retrieving Results for 2 : 9780385547932.
Retrieving Results for 3 : 9780593465066.
Retrieving Results for 4 : 9781982168438.
Retrieving Results for 5 : 9781538728628.
Retrieving Results for 6 : 9781101885680.
Retrieving Results for 7 : 9780593230572.
Retrieving Results for 8 : 9780358567363.
Retrieving Results for 9 : 9780062892355.
Retrieving Results for 10 : 9780063209787.
Retrieving Results for 11 : 9780593139134.
Retrieving Results for 12 : 9781984877925.
Retrieving Results for 13 : 9780525657743.
Retrieving Results for 14 : 9780063076099.
Retrieving Results for 15 : 9781984801258.
Retrieving Results for 16 : 9781982132118.
Retrieving Results for 17 : 9781510766808.
Retrieving Results for 18 : 9781637630594.
Retrieving Results for 19 : 9781982173760.
Retrieving Results for 20 : 9781984878106.
Retrieving Results for 21 : 9781984818416.
Retrieving Results for 22 : 9780593296844.
Retrieving Results fo

Retrieving Results for 165 : 9780063046641.
Retrieving Results for 166 : 9781734537727.
Title not found. Skipping...
Retrieving Results for 167 : 9781250784094.
Retrieving Results for 168 : 9780063092785.
Title not found. Skipping...
Retrieving Results for 169 : 9780935112962.
Retrieving Results for 170 : 9780593135204.
Retrieving Results for 171 : 9781980035961.
Title not found. Skipping...
Retrieving Results for 172 : 9781427201447.
Title not found. Skipping...
Retrieving Results for 173 : 9780063112261.
Title not found. Skipping...
Retrieving Results for 174 : 9780593290743.
Title not found. Skipping...
Retrieving Results for 175 : 9781603932158.
Title not found. Skipping...
Retrieving Results for 176 : 9781980060338.
Title not found. Skipping...
Retrieving Results for 177 : 9780062796233.
Title not found. Skipping...
Retrieving Results for 178 : 9781039406803.
Title not found. Skipping...
Retrieving Results for 179 : 9780593162392.
Title not found. Skipping...
Retrieving Results fo

Retrieving Results for 318 : 9781101947333.
Retrieving Results for 319 : 9780393881554.
Retrieving Results for 320 : 9781546090786.
Retrieving Results for 321 : 9781524746728.
Retrieving Results for 322 : 9781984821492.
Retrieving Results for 323 : 9780765387561.
Retrieving Results for 324 : 9780062942937.
Retrieving Results for 325 : 9780316540711.
Retrieving Results for 326 : 9781984820464.
Retrieving Results for 327 : 9781982128685.
Retrieving Results for 328 : 9781538719268.
Retrieving Results for 329 : 9781541730335.
Retrieving Results for 330 : 9781538736708.
Retrieving Results for 331 : 9780593298947.
Retrieving Results for 332 : 9780063001824.
Retrieving Results for 333 : 9780063018952.
Retrieving Results for 334 : 9781250178602.
Retrieving Results for 335 : 9780316420082.
Retrieving Results for 336 : 9780062834867.
Retrieving Results for 337 : 9781982175160.
Retrieving Results for 338 : 9780063049086.
Retrieving Results for 339 : 9780593505069.
Title not found. Skipping...
Ret

Title not found. Skipping...
Retrieving Results for 449 : 9781469092355.
Title not found. Skipping...
Retrieving Results for 450 : 9781452184715.
Title not found. Skipping...
Retrieving Results for 451 : 9780525528067.
Title not found. Skipping...
Retrieving Results for 452 : 9781549105401.
Title not found. Skipping...
Retrieving Results for 453 : 9780307987570.
Title not found. Skipping...
Retrieving Results for 454 : 9780593289358.
Title not found. Skipping...
Retrieving Results for 455 : 9781549142284.
Title not found. Skipping...
Retrieving Results for 456 : 9781549160769.
Title not found. Skipping...
Retrieving Results for 457 : 9780062477521.
Title not found. Skipping...
Retrieving Results for 458 : 9781774246320.
Title not found. Skipping...
Retrieving Results for 459 : 9780593168554.
Title not found. Skipping...
Retrieving Results for 460 : 9781250759788.
Title not found. Skipping...
Retrieving Results for 461 : 9781455814251.
Title not found. Skipping...
Retrieving Results for

Title not found. Skipping...
Retrieving Results for 563 : 9781600243929.
Title not found. Skipping...
Retrieving Results for 564 : 9781797103204.
Title not found. Skipping...
Retrieving Results for 565 : 9781980057673.
Title not found. Skipping...
Retrieving Results for 566 : 9781980021810.
Title not found. Skipping...
Retrieving Results for 567 : 9781549170843.
Title not found. Skipping...
Retrieving Results for 568 : DBKADBL050607.
Title not found. Skipping...
Retrieving Results for 569 : 9780593165768.
Title not found. Skipping...
Retrieving Results for 570 : 9781441817204.
Title not found. Skipping...
Retrieving Results for 571 : 9781508240013.
Title not found. Skipping...
Retrieving Results for 572 : 9780593212325.
Title not found. Skipping...
Retrieving Results for 573 : 9781250259981.
Title not found. Skipping...
Retrieving Results for 574 : 9780062878847.
Title not found. Skipping...
Retrieving Results for 575 : 9781797103167.
Title not found. Skipping...
Retrieving Results for

Title not found. Skipping...
Retrieving Results for 708 : A00B01KKOTNBW.
Title not found. Skipping...
Retrieving Results for 709 : 9781942832034.
Retrieving Results for 710 : 9780997552942.
Retrieving Results for 711 : 9781250132932.
Retrieving Results for 712 : 9781620972267.
Title not found. Skipping...
Retrieving Results for 713 : 9780698198067.
Retrieving Results for 714 : 9781595588197.
Title not found. Skipping...
Retrieving Results for 715 : 9780307886866.
Retrieving Results for 716 : 9781466836952.
Retrieving Results for 717 : 9781101608487.
Retrieving Results for 718 : 9781101595954.
Retrieving Results for 719 : 9781466867512.
Retrieving Results for 720 : 9780062463562.
Retrieving Results for 721 : 9781429952118.
Retrieving Results for 722 : 9780692527771.
Retrieving Results for 723 : 9780998244419.
Title not found. Skipping...
Retrieving Results for 724 : 9780698411340.
Retrieving Results for 725 : 9781942834205.
Retrieving Results for 726 : 9780698406216.
Retrieving Results 

Retrieving Results for 885 : 9780062441393.
Retrieving Results for 886 : 9780061859960.
Title not found. Skipping...
Retrieving Results for 887 : 9780061885075.
Retrieving Results for 888 : 9781476758756.
Retrieving Results for 889 : 9781101988978.
Retrieving Results for 890 : 9780062425478.
Retrieving Results for 891 : 9781501110375.
Retrieving Results for 892 : 9781945253157.
Retrieving Results for 893 : 9781250087119.
Retrieving Results for 894 : 9781612196336.
Retrieving Results for 895 : 9780062434784.
Retrieving Results for 896 : 9781942215462.
Retrieving Results for 897 : 9780062499042.
Retrieving Results for 898 : 9780718021771.
Retrieving Results for 899 : 9781588364098.
Retrieving Results for 900 : 9780062342867.
Retrieving Results for 901 : 9780440335160.
Retrieving Results for 902 : 9781476798912.
Retrieving Results for 903 : 9781588369390.
Retrieving Results for 904 : 9780698179967.
Retrieving Results for 905 : A00B01F3BYKTW.
Title not found. Skipping...
Retrieving Results

Retrieving Results for 1061 : 9780399160752.
Retrieving Results for 1062 : 9780062241450.
Retrieving Results for 1063 : 9780307460998.
Retrieving Results for 1064 : 9780547848419.
Retrieving Results for 1065 : 9780307592736.
Retrieving Results for 1066 : 9781594205347.
Retrieving Results for 1067 : 9780316017930.
Retrieving Results for 1068 : 9780446547574.
Retrieving Results for 1069 : 9780345523501.
Retrieving Results for 1070 : 9781476733951.
Retrieving Results for 1071 : 9780425259290.
Retrieving Results for 1072 : 9780547928173.
Retrieving Results for 1073 : 9781401324797.
Retrieving Results for 1074 : 9780812993929.
Retrieving Results for 1075 : 9780061967559.
Retrieving Results for 1076 : 9781455523559.
Retrieving Results for 1077 : 9781414319704.
Retrieving Results for 1078 : 9781451647037.
Retrieving Results for 1079 : 9780399158827.
Retrieving Results for 1080 : 9780446571821.
Retrieving Results for 1081 : 9781420118537.
Retrieving Results for 1082 : 9780786963645.
Retrieving

In [20]:
#viewing the updated dataframe
amazon_df = nyt_df

amazon_df.tail(150)

,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price
1087,16,9780393087727,Norton,WHITEY BULGER,Kevin Cullen and Shelley Murphy,Combined Print Nonfiction,2013-03-17,0393087727,4.6,479,25.61
1088,20,9780670026715,Penguin Group,THE VATICAN DIARIES,John Thavis,Combined Print Nonfiction,2013-03-17,B008EKOJZK,4.2,303,14.99
1089,15,9780307408853,Crown Publishing,IN THE GARDEN OF BEASTS,Erik Larson,Combined Print Nonfiction,2013-03-17,9124091596,,,
1090,17,9780307700667,Knopf Doubleday Publishing,GOING CLEAR,Lawrence Wright,Combined Print Nonfiction,2013-03-17,B00AYLFLCM,4.4,2111,0
1091,18,9781400067558,Random House Publishing,BEHIND THE BEAUTIFUL FOREVERS,Katherine Boo,Combined Print Nonfiction,2013-03-17,081297932X,4.3,3540,10.18
...,...,...,...,...,...,...,...,...,...,...,...
1232,18,9780312547745,St. Martin's Press,"THE WALKING DEAD, RISE OF THE GOVERNOR",Robert Kirkman and Jay Bonansinga,Combined Print Fiction,2012-11-04,B009RFHNB8,4.5,930,0
1233,15,9780399157707,Penguin Group,MAD RIVER,John Sandford,Combined Print Fiction,2012-11-04,B0085DP6EY,4.6,3011,9.99
1234,12,9781439152805,Atria Books,THE SECRET KEEPER,Kate Morton,Combined Print Fiction,2012-11-04,B009R5Q51W,4.5,7219,0
1235,2,9780345504982,Random House Publishing,THE TWELVE,Justin Cronin,Combined Print Fiction,2012-11-04,B009RFGP8K,4.5,4496,0


In [40]:
#cleaning the dataframe
#filling any blank cells with NaN
clean_amazon_df = amazon_df.replace(r'^\s*$', np.nan, regex=True)

clean_amazon_df.tail(150)

,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price
1087,16,9780393087727,Norton,WHITEY BULGER,Kevin Cullen and Shelley Murphy,Combined Print Nonfiction,2013-03-17,0393087727,4.6,479.0,25.61
1088,20,9780670026715,Penguin Group,THE VATICAN DIARIES,John Thavis,Combined Print Nonfiction,2013-03-17,B008EKOJZK,4.2,303.0,14.99
1089,15,9780307408853,Crown Publishing,IN THE GARDEN OF BEASTS,Erik Larson,Combined Print Nonfiction,2013-03-17,9124091596,NaN,NaN,NaN
1090,17,9780307700667,Knopf Doubleday Publishing,GOING CLEAR,Lawrence Wright,Combined Print Nonfiction,2013-03-17,B00AYLFLCM,4.4,2111.0,0.00
1091,18,9781400067558,Random House Publishing,BEHIND THE BEAUTIFUL FOREVERS,Katherine Boo,Combined Print Nonfiction,2013-03-17,081297932X,4.3,3540.0,10.18
...,...,...,...,...,...,...,...,...,...,...,...
1232,18,9780312547745,St. Martin's Press,"THE WALKING DEAD, RISE OF THE GOVERNOR",Robert Kirkman and Jay Bonansinga,Combined Print Fiction,2012-11-04,B009RFHNB8,4.5,930.0,0.00
1233,15,9780399157707,Penguin Group,MAD RIVER,John Sandford,Combined Print Fiction,2012-11-04,B0085DP6EY,4.6,3011.0,9.99
1234,12,9781439152805,Atria Books,THE SECRET KEEPER,Kate Morton,Combined Print Fiction,2012-11-04,B009R5Q51W,4.5,7219.0,0.00
1235,2,9780345504982,Random House Publishing,THE TWELVE,Justin Cronin,Combined Print Fiction,2012-11-04,B009RFGP8K,4.5,4496.0,0.00


In [42]:
#dropping any books without Amazon data
clean_amazon_df2=clean_amazon_df.dropna(axis=0, how="any")

#viewing the cleaned dataframe
clean_amazon_df2.tail(150)


,rank,primary_isbn13,publisher,title,author,category,pub_date,Amazon Product ID,Amazon Rating,Amazon Rating Total,Amazon Price
1084,20,9780451233967,Penguin Group,STAY CLOSE,Harlan Coben,Combined Print Fiction,2013-03-17,B005GSYXYK,4.4,3238.0,7.99
1085,13,9780062078117,HarperCollins Publishers,IMMORTAL EVER AFTER,Lynsay Sands,Combined Print Fiction,2013-03-17,B00AV6FD64,4.8,900.0,7.99
1086,16,9780758280343,Kensington Publishing,RED VELVET CUPCAKE MURDER,Joanne Fluke,Combined Print Fiction,2013-03-17,0758280343,4.6,1460.0,26.94
1087,16,9780393087727,Norton,WHITEY BULGER,Kevin Cullen and Shelley Murphy,Combined Print Nonfiction,2013-03-17,0393087727,4.6,479.0,25.61
1088,20,9780670026715,Penguin Group,THE VATICAN DIARIES,John Thavis,Combined Print Nonfiction,2013-03-17,B008EKOJZK,4.2,303.0,14.99
...,...,...,...,...,...,...,...,...,...,...,...
1232,18,9780312547745,St. Martin's Press,"THE WALKING DEAD, RISE OF THE GOVERNOR",Robert Kirkman and Jay Bonansinga,Combined Print Fiction,2012-11-04,B009RFHNB8,4.5,930.0,0.00
1233,15,9780399157707,Penguin Group,MAD RIVER,John Sandford,Combined Print Fiction,2012-11-04,B0085DP6EY,4.6,3011.0,9.99
1234,12,9781439152805,Atria Books,THE SECRET KEEPER,Kate Morton,Combined Print Fiction,2012-11-04,B009R5Q51W,4.5,7219.0,0.00
1235,2,9780345504982,Random House Publishing,THE TWELVE,Justin Cronin,Combined Print Fiction,2012-11-04,B009RFGP8K,4.5,4496.0,0.00


In [43]:
#counting the number of rows in the full dataframe, without blank rows
clean_amazon_df2.count()

rank                   860
primary_isbn13         860
publisher              860
title                  860
author                 860
category               860
pub_date               860
Amazon Product ID      860
Amazon Rating          860
Amazon Rating Total    860
Amazon Price           860
dtype: int64

In [ ]:
clean_amazon_df2.duplicated(subset=['ISBN']).sum()

In [ ]:
clean_amazon_df2.drop_duplicates(subset=['ISBN'],inplace=True)

In [44]:
#writing the file to a csv
clean_amazon_df2.to_csv("output/combined_book.csv",index=False)